In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

# Defining DELTA at reach mark 

In [2]:
#Creating a function that define the delta at the reach mark - delta meaning that the first boat gets the delta
#0 and then the next boats have the delta between the time where they round the mark and the leaders time. 

def reachmark_delta(day, sequence):
    """ This functions calculates 
    det delta value from the first
    boat rounding the reach mark 
    to every boat rounding
    Input: .csv files
    -------------------------------
    Output: dataframe included delta values. 
    """
    
    delta_list = pd.DataFrame()
#     for race in races: 
    for start,end in sequence:
        race = (day[(day['DATETIME']>=start) & (day['DATETIME']<=end)]).reset_index(drop = True)
        
        #make a list of the delta times in sec at the reach mark 
        delta = []
        # Make a list of the timestamps when at reach mark 
        timestamps = []
        #Defining the list of boats that has pasted the mark
        L2 = [] 
        #define unique race
        unq_race =[]

        number_of_boats =[1,2,3,4,5,6,7,8,9]

        for timestamp in range(0,len(race)):
            if race['TRK_LEG_NUM_unk'][timestamp] == 2:
                for i in range(0,len(number_of_boats)):
                    if list(race['POSITION_AT_LAST_MARK*'])[timestamp] == number_of_boats[i]:
                        if list(race['BOAT'])[timestamp] not in L2: 
                            L2.append(list(race['BOAT'])[timestamp])
                            time = race['DATETIME'][timestamp]
                            timestamps.append(time)
                            deltaa = (timestamps[0]-timestamps[-1]).total_seconds()
                            #making delta exponential to avoid dealing with 0 values as y-variable
#                             delta.append(np.exp(-deltaa))
                            delta.append(deltaa)
                            unq_race.append(race['TRK_RACE_NUM_unk'][timestamp])



            delta_list_midlertidig = pd.DataFrame(np.column_stack([unq_race, L2, delta]),columns=['TRK_RACE_NUM_unk','BOAT', 'DELTA'])
        delta_list=delta_list.append(delta_list_midlertidig,ignore_index=True)
    
##----First we need to create a unique column:
    delta_list['joincolumn'] = delta_list.apply(lambda row: row.TRK_RACE_NUM_unk + row.BOAT , axis=1)
##----Change float64 to string to be able to add the columns for the unique column
    day['TRK_RACE_NUM_unk'] = day['TRK_RACE_NUM_unk'].astype(str)
    day['TRK_LEG_NUM_unk'] = day['TRK_LEG_NUM_unk'].astype(str)
    day['joincolumn'] = day.apply(lambda row: row.TRK_RACE_NUM_unk + row.BOAT , axis=1)


    #Now merging df and DELTA
    df1 = pd.merge(day, delta_list, on='joincolumn', how='left')
    df = df1.drop(['TRK_RACE_NUM_unk_y','BOAT_y','joincolumn'], axis=1)
    df.rename(columns={ 'TRK_RACE_NUM_unk_x': 'TRK_RACE_NUM_unk','BOAT_x':'BOAT'}, inplace=True)
    df['TRK_RACE_NUM_unk'] = df['TRK_RACE_NUM_unk'].astype(float)
    df['TRK_LEG_NUM_unk'] = df['TRK_LEG_NUM_unk'].astype(float)



    return(df)